In [17]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import spearmanr
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

These are all available features provided by the mappings_with_conductance.csv dataset.

# Data Preprocessing

In [2]:
ELECPHYS_FEATURES = ['decay_CaDynamics_E2_axonal', 'decay_CaDynamics_E2_somatic', 'e_pas_axonal', 'e_pas_somatic', 'gCa_LVAstbar_Ca_LVAst_axonal', 
                    'gCa_LVAstbar_Ca_LVAst_somatic', 'gCabar_Ca_axonal', 'gCabar_Ca_somatic', 'gIhbar_Ih_dend', 'gImbar_Im_axonal', 'gImbar_Im_dend', 
                    'gImbar_Im_somatic', 'gK_Pstbar_K_Pst_axonal', 'gK_Pstbar_K_Pst_somatic', 'gK_Tstbar_K_Tst_axonal', 'gK_Tstbar_K_Tst_dend', 
                    'gK_Tstbar_K_Tst_somatic', 'gNaTa_tbar_NaTa_t_axonal', 'gNaTs2_tbar_NaTs2_t_dend', 'gNaTs2_tbar_NaTs2_t_somatic', 'gNap_Et2bar_Nap_Et2_axonal', 
                    'gNap_Et2bar_Nap_Et2_dend',  'gSK_E2bar_SK_E2_axonal', 'gSK_E2bar_SK_E2_somatic', 'gSKv3_1bar_SKv3_1_axonal', 'gSKv3_1bar_SKv3_1_dend', 
                    'gSKv3_1bar_SKv3_1_somatic', 'g_pas_axonal', 'g_pas_somatic', 'gamma_CaDynamics_E2_axonal', 'gamma_CaDynamics_E2_somatic', 'g_pas_dend', 
                    'gNap_Et2bar_Nap_Et2_somatic']

MORPH_FEATURES = ['length_soma', 'avg_length_dendrite', 'avg_length_axon', 'total_length_dendrite', 'total_length_axon', 
                    'length_dendrite_var', 'length_axon_var', 'count_axon', 'count_dendrite', 'avg_diameter_dendrite',
                    'avg_diameter_axon', 'var_diameter_dendrite', 'var_diameter_axon', 'median_length_dendrite', 'median_length_axon',
                    'median_diameter_dendrite', 'median_diameter_axon', 'var_children_axon', 'var_children_dendrite', 'median_children_axon',
                    'median_children_dendrite', 'mean_children_axon', 'mean_children_dendrite']

ALL_FEATURES = ELECPHYS_FEATURES + MORPH_FEATURES
FEATURES_TO_RANK = ELECPHYS_FEATURES + MORPH_FEATURES

In [19]:
DCA_THRESHOLD_PER_CELL_UNNORM_95 = pd.read_csv('data/dca_threshold_per_cell_unnorm_950.0.csv')
# DCA_THRESHOLD_PER_CELL_UNNORM_68 = pd.read_csv('data/dca_threshold_per_cell_unnorm_680.0.csv')

DCA_THRESHOLD_PER_CELL_UNNORM = DCA_THRESHOLD_PER_CELL_UNNORM_95

MAPPINGS_WITH_CONDUCTANCE = pd.read_csv('data/mappings_with_conductance.csv').dropna()

# normalize the numerical columns
numeric_columns = MAPPINGS_WITH_CONDUCTANCE.select_dtypes(include=['int64', 'float64']).columns
scaler = StandardScaler()
MAPPINGS_WITH_CONDUCTANCE[numeric_columns] = scaler.fit_transform(MAPPINGS_WITH_CONDUCTANCE[numeric_columns])

MAPPINGS_WITH_CONDUCTANCE

,Unnamed: 0,bbp_name,long_name,m_type,e_type,binned_complexity,horizontal_complexity,level,length_soma,avg_length_dendrite,...,gSKv3_1bar_SKv3_1_somatic,g_pas_axonal,g_pas_dend,g_pas_somatic,gamma_CaDynamics_E2_axonal,gamma_CaDynamics_E2_somatic,gkbar_KdShu2007_dend,gkbar_KdShu2007_somatic,gkbar_StochKv_dend,gkbar_StochKv_somatic
43,-1.718249,bbp054,L23_PC_cADpyr229_1,PC,cAD,-0.620027,-0.896811,5.334312,-1.495131,-0.595196,...,-2.073103,0.788049,-2.117582e-22,0.544936,2.464293,-0.432104,1.084202e-19,-5.421011e-20,1.27704,-0.505418
44,-1.690536,bbp055,L23_SBC_bNAC219_1,SBC,bNAC,-0.676111,-0.444379,5.334312,-1.495131,-0.595196,...,1.788854,-1.830660,-2.117582e-22,0.816198,-0.735951,-0.560308,1.084202e-19,-5.421011e-20,1.27704,-0.505418
45,-1.662822,bbp056,L23_SBC_cACint209_1,SBC,cAC,1.219604,2.381921,5.334312,-1.495131,-0.595196,...,-0.549444,-0.155905,-2.117582e-22,-1.595023,-1.000109,0.659578,1.084202e-19,-5.421011e-20,1.27704,-0.505418
46,-1.635108,bbp057,L23_SBC_dNAC222_1,SBC,dNAC,-0.252734,-0.107011,5.334312,-1.495131,-0.595196,...,-0.196449,0.788049,-2.117582e-22,0.544936,1.105033,-0.517574,1.084202e-19,-5.421011e-20,1.27704,-0.505418
47,-1.607395,bbp060,L4_BP_bNAC219_1,BP,bNAC,-0.859069,-1.035333,-0.490511,-1.014980,-0.682943,...,1.788854,-1.830660,-2.117582e-22,0.816198,-0.735951,-0.560308,1.084202e-19,-5.421011e-20,1.27704,-0.505418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,1.607395,bbp203,L6_SBC_cACint209_1,SBC,cAC,-0.777597,-0.845423,0.122628,-0.272034,1.430923,...,-0.549444,-0.155905,-2.117582e-22,-1.595023,-1.000109,0.659578,1.084202e-19,-5.421011e-20,1.27704,-0.505418
164,1.635108,bbp204,L6_SBC_dNAC222_1,SBC,dNAC,-0.727880,-0.843189,0.122628,-0.272034,1.430923,...,-0.196449,0.788049,-2.117582e-22,0.544936,1.105033,-0.517574,1.084202e-19,-5.421011e-20,1.27704,-0.505418
165,1.662822,bbp205,L6_TPC_L1_cADpyr231_1,TPC,cAD,-0.538378,-0.409749,0.122628,-0.272034,1.430923,...,-2.357792,0.788049,-2.117582e-22,0.544936,-0.737019,1.366645,1.084202e-19,-5.421011e-20,1.27704,-0.505418
166,1.690536,bbp206,L6_TPC_L4_cADpyr231_1,TPC,cAD,-0.843197,-0.779514,0.122628,-0.272034,1.430923,...,-2.357792,0.788049,-2.117582e-22,0.544936,-0.737019,1.366645,1.084202e-19,-5.421011e-20,1.27704,-0.505418


# 1. Results using Random Forest

Objective is to predict the number of DCA components required to cross $n\%$ of the threshold where $n=68,95$, as well as the predictive information at that number of dynamical components. 

In [20]:
def features_with_pi_from_dataframe(features, dataframe=MAPPINGS_WITH_CONDUCTANCE, threshold_percent=0.95):
    dataframe_with_features = dataframe[['bbp_name'] + FEATURES_TO_RANK].rename(columns={'bbp_name': 'cell_name'})
    threshold_info = None
    if threshold_percent == 0.95:
        threshold_info = DCA_THRESHOLD_PER_CELL_UNNORM_95
    elif threshold_percent == 0.68:
        threshold_info = DCA_THRESHOLD_PER_CELL_UNNORM_68
    
    return dataframe_with_features.merge(threshold_info[['cell_name', 'pi']], on='cell_name', how='inner')

def features_with_dca_from_dataframe(features, dataframe=MAPPINGS_WITH_CONDUCTANCE, threshold_percent=0.95):
    dataframe_with_features = dataframe[['bbp_name'] + FEATURES_TO_RANK].rename(columns={'bbp_name': 'cell_name'})
    threshold_info = None
    if threshold_percent == 0.95:
        threshold_info = DCA_THRESHOLD_PER_CELL_UNNORM_95
    elif threshold_percent == 0.68:
        threshold_info = DCA_THRESHOLD_PER_CELL_UNNORM_68
    
    return dataframe_with_features.merge(threshold_info[['cell_name', 'dca_level']], on='cell_name', how='inner')

FEATURES_WITH_PI = features_with_pi_from_dataframe(ALL_FEATURES)
FEATURES_WITH_DCA = features_with_dca_from_dataframe(ALL_FEATURES)

## 1.1 Grid Search for optimal parameters

In [21]:
def random_forest_predictor(features_with_predicted_val, val_to_predict='dca_level', random_state=1, num_estimators=100, split=0.2):
    X = features_with_predicted_val.drop([val_to_predict], axis=1)  
    y = features_with_predicted_val[val_to_predict]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split, random_state=random_state)
    X_train = X_train.drop(['cell_name'], axis=1)
    X_test = X_test.drop(['cell_name'], axis=1)
    
    rf_model = RandomForestRegressor(n_estimators=num_estimators, random_state=random_state)
    rf_model.fit(X_train, y_train)   
    
    y_pred = rf_model.predict(X_test)
    
    return {'MSE' : mean_squared_error(y_test, y_pred), 'R2' : r2_score(y_test, y_pred), 'model' : rf_model,
           'y_pred' : y_pred, 'y_test' : np.array(y_test)}

def grid_search_random_forest_predictor(features_with_predicted_val, val_to_predict='dca_level', random_state=1,
                                        estimators=[25, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500], num_folds=5, split=0.2):
    param_grid = {
        'n_estimators': estimators
    }
    
    X = features_with_predicted_val.drop([val_to_predict], axis=1)  
    y = features_with_predicted_val[val_to_predict]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split, random_state=random_state)
    X_train = X_train.drop(['cell_name'], axis=1)
    X_test = X_test.drop(['cell_name'], axis=1)
    X = X.drop(['cell_name'], axis=1)
    
    rf_model = grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=random_state), param_grid=param_grid, cv=num_folds)
    rf_model.fit(X, y)   
    
    best_params = rf_model.best_params_
    final_rf_model = RandomForestRegressor(**best_params, random_state=random_state)
    final_rf_model.fit(X_train, y_train)
    
    y_pred = final_rf_model.predict(X_test)
    
    return {'MSE' : mean_squared_error(y_test, y_pred), 'R2' : r2_score(y_test, y_pred), 'model' : final_rf_model,
           'y_pred' : y_pred, 'y_test' : np.array(y_test)}

In [22]:
MODEL_PARAMS = grid_search_random_forest_predictor(FEATURES_WITH_DCA)
RF_MODEL_PI = MODEL_PARAMS['model']

print(MODEL_PARAMS)

{'MSE': 185.1119587238962, 'R2': 0.2751123142124562, 'model': RandomForestRegressor(n_estimators=50, random_state=1), 'y_pred': array([ 49.33856854,  39.2178803 ,  49.33856854,  36.1648138 ,
        39.2178803 ,  39.2178803 ,  36.00474672,  45.83215873,
        45.83215873,  35.12672078,  35.31562882,  39.2178803 ,
        45.83215873,  45.83215873,  45.83215873,  52.38161939,
        36.1648138 ,  36.00474672, 103.86533333,  49.6286178 ,
        49.33856854, 103.86533333,  52.38161939,  49.33856854,
        36.00474672]), 'y_test': array([39., 36., 52., 18., 40., 43., 35., 34., 32., 28., 51., 22., 64.,
       45., 43., 42., 20., 62., 78., 53., 58., 75., 53., 62., 26.])}


In [24]:
grid_search_random_forest_predictor(features_with_dca_from_dataframe(ELECPHYS_FEATURES + MORPH_FEATURES))

{'MSE': 185.1119587238962,
 'R2': 0.2751123142124562,
 'model': RandomForestRegressor(n_estimators=50, random_state=1),
 'y_pred': array([ 49.33856854,  39.2178803 ,  49.33856854,  36.1648138 ,
         39.2178803 ,  39.2178803 ,  36.00474672,  45.83215873,
         45.83215873,  35.12672078,  35.31562882,  39.2178803 ,
         45.83215873,  45.83215873,  45.83215873,  52.38161939,
         36.1648138 ,  36.00474672, 103.86533333,  49.6286178 ,
         49.33856854, 103.86533333,  52.38161939,  49.33856854,
         36.00474672]),
 'y_test': array([39., 36., 52., 18., 40., 43., 35., 34., 32., 28., 51., 22., 64.,
        45., 43., 42., 20., 62., 78., 53., 58., 75., 53., 62., 26.])}

## 1.2 Feature Importance with Spearman

In [25]:
def get_feature_spearman_corr(dataframe, model, val_to_predict='dca_level'):
    X = dataframe.drop([val_to_predict], axis=1)  
    X = X.drop(['cell_name'], axis=1)
    y = dataframe[val_to_predict]
    spearman_rank_correlation = {}
    
    y_pred = model.predict(X)
    
    for column in X.columns:
        correlation, _ = spearmanr(X[column], y_pred)
        spearman_rank_correlation[column] = correlation
    
    ranked_features_df = pd.DataFrame(spearman_rank_correlation.items(), columns=['Feature', 'Spearman_Correlation'])
    ranked_features_df = ranked_features_df.sort_values(by='Spearman_Correlation', ascending=False)
    return ranked_features_df

In [26]:
get_feature_spearman_corr(FEATURES_WITH_DCA, RF_MODEL_PI)

/usr/lib/python3/dist-packages/scipy/stats/_stats_py.py:4529: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


,Feature,Spearman_Correlation
12,gK_Pstbar_K_Pst_axonal,0.776813
22,gSK_E2bar_SK_E2_axonal,0.775277
19,gNaTs2_tbar_NaTs2_t_somatic,0.772434
30,gamma_CaDynamics_E2_somatic,0.761583
24,gSKv3_1bar_SKv3_1_axonal,0.702918
1,decay_CaDynamics_E2_somatic,0.573690
0,decay_CaDynamics_E2_axonal,0.554897
5,gCa_LVAstbar_Ca_LVAst_somatic,0.520169
6,gCabar_Ca_axonal,0.434790
18,gNaTs2_tbar_NaTs2_t_dend,0.434790


# 2. Results using XGBoost

In [27]:
def xgb_predictor(features_with_predicted_val, val_to_predict='dca_level', random_state=1, num_estimators=100, split=0.2):
    X = features_with_predicted_val.drop([val_to_predict], axis=1)  
    y = features_with_predicted_val[val_to_predict]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split)
    X_train = X_train.drop(['cell_name'], axis=1)
    X_test = X_test.drop(['cell_name'], axis=1)
    X = X.drop(['cell_name'], axis=1)
    
    params_xgb = {
        'objective': 'reg:squarederror',  # Use 'reg:squarederror' for regression tasks
        'learning_rate': 0.15,
        'max_depth': 30,
        'min_child_weight': 50,
        'gamma': 0.1,
        'subsample': 0.8,
        'colsample_bytree': 0.9,
        'seed': random_state,
        'n_estimators': 500,
    }
    
    # params_xgb = {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 50, 'subsample': 1.0}
    
    xgb_model = xgb.XGBRegressor(**params_xgb)
    xgb_model.fit(X_train, y_train)   
    
    y_pred = xgb_model.predict(X)
    
    # print(mean_squared_error(y_train, xgb_model.predict(X_train)))
    # print(r2_score(y_train, xgb_model.predict(X_train)))
        
    return {'MSE' : mean_squared_error(y, y_pred), 'R2' : r2_score(y, y_pred), 'model' : xgb_model,
           'y_pred' : y_pred, 'y_test' : np.array(y_test)}

xgb_predictor(FEATURES_WITH_DCA)

{'MSE': 276.6445869248453,
 'R2': -0.0015258786058871543,
 'model': XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.1, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.15, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=30, max_leaves=None,
              min_child_weight=50, missing=nan, monotone_constraints=None,
              n_estimators=500, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...),
 'y_pred': array([44.313217, 44.313217, 44.313217, 44.313217, 44.313217, 44.313217,
        44.313217, 44.313217, 44.313217, 44.313217, 44.313217, 44.313217,
        44.313217, 44.313217

In [28]:
def grid_search_xgb_predictor(features_with_predicted_val, val_to_predict='dca_level', random_state=1, 
                              estimators=[400, 450, 500], num_folds=4):
    params_xgb = {
        'learning_rate': [0.15, 0.2, 0.25],
        'n_estimators': estimators,
        'max_depth': [10, 30, 50],
        'min_child_weight': [10, 30, 50],
        'subsample': [0.7, 0.8, 0.9],
        'colsample_bytree': [0.8, 0.9, 1.0],
        'gamma': [0.3, 1, 15]
    }
    
    X = features_with_predicted_val.drop([val_to_predict], axis=1)  
    y = features_with_predicted_val[val_to_predict]
    X = X.drop(['cell_name'], axis=1)
    
    xgb_model = GridSearchCV(
        estimator=xgb.XGBRegressor(),
        param_grid=params_xgb,
        scoring='r2',  # Using negative MSE since GridSearchCV maximizes the scoring function
        cv=num_folds,  
        n_jobs=-1  
    )
    
    grid_result = xgb_model.fit(X, y)
        
    best_model = grid_result.best_estimator_
    y_pred = best_model.predict(X)

    return {'MSE' : mean_squared_error(y, y_pred), 'R2' : r2_score(y, y_pred), 'model' : best_model, 'params' : grid_result.best_params_}

In [29]:
grid_search_xgb_predictor(FEATURES_WITH_DCA, val_to_predict='dca_level')

{'MSE': 119.23537669805685,
 'R2': 0.568336699677168,
 'model': XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=1.0, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.3, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.15, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=30, missing=nan, monotone_constraints=None,
              n_estimators=400, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...),
 'params': {'colsample_bytree': 1.0,
  'gamma': 0.3,
  'learning_rate': 0.15,
  'max_depth': 10,
  'min_child_weight': 30,
  'n_estimators': 400,
  'subsample': 0.9}}

# Results using Lasso Regression

In [ ]:
def grid_search_lasso_predictor(features_with_predicted_val, val_to_predict='dca_level', random_state=1, 
                              estimators=[400, 450, 500], num_folds=4):
    params_xgb = {
        'learning_rate': [0.15, 0.2, 0.25],
        'n_estimators': estimators,
        'max_depth': [10, 30, 50],
        'min_child_weight': [10, 30, 50],
        'subsample': [0.7, 0.8, 0.9],
        'colsample_bytree': [0.8, 0.9, 1.0],
        'gamma': [0.3, 1, 15],
        'alpha': [0.01, 0.1, 1.0]
    }
    
    X = features_with_predicted_val.drop([val_to_predict], axis=1)  
    y = features_with_predicted_val[val_to_predict]
    X = X.drop(['cell_name'], axis=1)
    
    xgb_model = GridSearchCV(
        estimator=xgb.XGBRegressor(),
        param_grid=params_xgb,
        scoring='r2',  # Using negative MSE since GridSearchCV maximizes the scoring function
        cv=num_folds,  
        n_jobs=-1  
    )
    
    grid_result = xgb_model.fit(X, y)
        
    best_model = grid_result.best_estimator_
    y_pred = best_model.predict(X)

    return {'MSE' : mean_squared_error(y, y_pred), 'R2' : r2_score(y, y_pred), 'model' : best_model, 'params' : grid_result.best_params_}